In [1]:
m1, m2, m3 = 2 * (10 ** 30), 1.9 * (10 ** 27), 1.5 * (10 ** 23)
G = 6.67 * (10 ** -11)
m0 = 75
mt = 4950
T = 1600
koef = 0.001
u = 3170
Rpl = 71500 * (10 ** 3)
H = 900 * (10 ** 3)
Rsat = 2634 * (10 ** 3)


In [5]:
from scipy.integrate import solve_ivp
import numpy as np

In [6]:
def m(t):
    if t >= T:
        return m0
    else:
        return (m0 + mt) / (1 - koef) - mt * t / T


def dm(t):
    if t >= T:
        return 0
    else:
        return -mt / T



In [7]:
def F2(t, y):
    rx, ry, r13x, r13y, vx, vy, v3x, v3y = y
    r = np.sqrt(rx**2 + ry**2)
    v = np.sqrt(vx**2 + vy**2)
    r2 = np.sqrt((rx - r12x)**2 + (ry - r12y)**2)
    r3 = np.sqrt((rx - r13x)**2 + (ry - r13y)**2)
    r13 = np.sqrt(r13x**2 + r13y**2)
    r23x = (r13x - r12x)
    r23y = (r13y - r12y)
    r23 = np.sqrt((r13x - r12x)**2 + (r13y - r12y)**2)

    return np.array([vx, vy, v3x, v3y,
                     (-u * dm(t) / m(t) * vx / v + G * (-m1 * rx / (r**3) - m2 * (rx - r12x) / (r2**3) - m3 * (rx - r13x) / (r3**3))),
                     (-u * dm(t) / m(t) * vy / v + G * (-m1 * ry / (r**3) - m2 * (ry - r12y) / (r2**3) - m3 * (ry - r13y) / (r3**3))),
                     G * (-(m1 / (r13**3) * r13x) - (m2 / (r23**3) * r23x)),
                     G * (-(m1 / (r13**3) * r13y) - (m2 / (r23**3) * r23y))
                     ])

In [8]:
def z2(r13x0,r13y0,v3x0,v3y0,tau,r12x0,r12y0,fi,m):
    global r12x,r12y
    global mt
    mt=m
    r12x=r12x0
    r12y=r12y0
    r3x=r13x0-r12x
    r3y=r13y0-r12y
    r3=np.sqrt(r3x**2+r3y**2)
    v0=1.0*np.sqrt(G*m2/(Rpl+H))
    rx0=(Rpl+H)*(r3x*np.cos(fi)-r3y*np.sin(fi))/r3
    ry0=(Rpl+H)*(r3x*np.sin(fi)+r3y*np.cos(fi))/r3
    r0=np.sqrt(rx0**2+ry0**2)
    vx0=-v0*ry0/r0
    vy0=v0*rx0/r0
    rx0=r12x+rx0
    ry0=r12y+ry0
    s0=[rx0,ry0,r13x0,r13y0,vx0,vy0,v3x0,v3y0]
    t_span=(0,306720)
    #t_span=(0,1442880*2)
    satel=solve_ivp(F2,t_span,s0, max_step=tau, atol=1, rtol=1,method="RK45")
    return satel


In [ ]:
import random
import math
tau = 0.1
a1 = random.random() * math.pi * 2
mt1 = random.randint(1, 20000)
y=z2(-166864782814.52066, -751687780697.0953, 10608.237381615552, -2321.8734688953678, tau,
                           -166632056766.0584, -750639289651.4109, a1, mt1)

In [ ]:
y